<a href="https://colab.research.google.com/github/ammarshse/testrepos/blob/main/project%20traf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
{
  "nbformat": 4,
  "nbformat_minor": 0,
  "metadata": {
    "colab": {
      "name": "PROJECT_LAST.ipynb",
      "provenance": [],
      "collapsed_sections": []
    },
    "kernelspec": {
      "name": "python3",
      "display_name": "Python 3"
    },
    "language_info": {
      "name": "python"
    },
    "accelerator": "GPU"
  },
  "cells": [
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "cKYMitFbgsvS"
      },
      "source": [
        "Клонируем репу"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "DVaeLq3Gg4hw"
      },
      "source": [
        "!sudo rm -rf /content/project"
      ],
      "execution_count": 1,
      "outputs": []
    },
    {
      "cell_type": "code",
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "o28E8GYVgphA",
        "outputId": "00fb8ba8-6382-4112-90d6-59ab06cb27ae"
      },
      "source": [
        "%cd /content\n",
        "!git clone https://github.com/STASVLAD/Tips-and-Tricks-in-DL project"
      ],
      "execution_count": 2,
      "outputs": [
        {
          "output_type": "stream",
          "text": [
            "/content\n",
            "Cloning into 'project'...\n",
            "remote: Enumerating objects: 264, done.\u001b[K\n",
            "remote: Counting objects: 100% (264/264), done.\u001b[K\n",
            "remote: Compressing objects: 100% (215/215), done.\u001b[K\n",
            "remote: Total 264 (delta 87), reused 147 (delta 31), pack-reused 0\u001b[K\n",
            "Receiving objects: 100% (264/264), 6.92 MiB | 16.79 MiB/s, done.\n",
            "Resolving deltas: 100% (87/87), done.\n"
          ],
          "name": "stdout"
        }
      ]
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "JYTmKQ1N8lSd"
      },
      "source": [
        "import os\n",
        "import cv2\n",
        "import time\n",
        "import json\n",
        "import torch\n",
        "import pandas as pd\n",
        "import numpy as np\n",
        "from google.colab import drive\n",
        "from IPython.display import clear_output\n",
        "from project.modules.detect_color import detect_color\n",
        "from project.modules.affect_check import affect_check\n",
        "from project.modules.check_coords import check_coords\n",
        "from project.modules.get_fps import get_fps"
      ],
      "execution_count": 3,
      "outputs": []
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "OZu3KKtEcgd1"
      },
      "source": [
        "Скачиваем модель"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "Lzas6j-ocQSB",
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "outputId": "f59175ac-8b91-4067-aa71-1a0c50bb1685"
      },
      "source": [
        "!gdown --id 1waoOULCmz1PA0K4pfSkTVHsCe9GYxzte -O /content/model.pt # Alex.pt"
      ],
      "execution_count": 4,
      "outputs": [
        {
          "output_type": "stream",
          "text": [
            "Downloading...\n",
            "From: https://drive.google.com/uc?id=1waoOULCmz1PA0K4pfSkTVHsCe9GYxzte\n",
            "To: /content/model.pt\n",
            "496MB [00:01, 265MB/s]\n"
          ],
          "name": "stdout"
        }
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "lTi2djabdMcd"
      },
      "source": [
        "Устанавливаем необходимые зависимости"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "ZO5ScG1UdAcR",
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "outputId": "655ddfd0-4966-40d0-f08f-bc17e93f6af2"
      },
      "source": [
        "%cd /content/project/yolov3\n",
        "%pip install -qr requirements.txt \n",
        "\n",
        "print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))"
      ],
      "execution_count": 5,
      "outputs": [
        {
          "output_type": "stream",
          "text": [
            "/content/project/yolov3\n",
            "Setup complete. Using torch 1.8.1+cu101 _CudaDeviceProperties(name='Tesla P4', major=6, minor=1, total_memory=7611MB, multi_processor_count=20)\n"
          ],
          "name": "stdout"
        }
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "EaHkbNbzddv_"
      },
      "source": [
        "Создаем необходимые временные папки и скачиваем видео "
      ]
    },
    {
      "cell_type": "code",
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "RU6n7dhsDn3w",
        "outputId": "0dc40b28-25f9-4b29-92ea-98858d9db998"
      },
      "source": [
        "drive.mount('/content/drive')"
      ],
      "execution_count": 6,
      "outputs": [
        {
          "output_type": "stream",
          "text": [
            "Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount(\"/content/drive\", force_remount=True).\n"
          ],
          "name": "stdout"
        }
      ]
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "w_QY8tTRQag5"
      },
      "source": [
        "!sudo rm -rf /content/images\n",
        "!sudo rm -rf /content/project/yolov3/runs\n",
        "!sudo rm -rf /content/output\n",
        "!sudo rm -rf /content/video.mp4"
      ],
      "execution_count": 7,
      "outputs": []
    },
    {
      "cell_type": "code",
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "kgVZvx4i62C8",
        "outputId": "d306f22a-b425-4aa5-f3e9-9267539e0ded"
      },
      "source": [
        "%cd /content\n",
        "!mkdir images\n",
        "!mkdir output"
      ],
      "execution_count": 8,
      "outputs": [
        {
          "output_type": "stream",
          "text": [
            "/content\n"
          ],
          "name": "stdout"
        }
      ]
    },
    {
      "cell_type": "code",
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "Vuodxd0BuNek",
        "outputId": "0696e98b-7587-45a7-9ece-9dab47518361"
      },
      "source": [
        "# !gdown --id 1E3zsggFnJanSoa-A28WcbUwll8MW0PMc -O /content/video.mp4 # vide 3\n",
        "# !gdown --id 1kh34S1LEW7eMB48hCsCukUXWvWgl2HEI -O /content/video.mp4 #video 1\n",
        "# !gdown --id 1c4AOUv43CGOH3qR9axkN7J30s0TlAHbT -O /content/video.mp4 #video 0\n",
        "!gdown --id 19-HG9CebZgVmLw_0yZMMeF9f6eoTqCYA -O /content/video.mp4 #video 2"
      ],
      "execution_count": 9,
      "outputs": [
        {
          "output_type": "stream",
          "text": [
            "Downloading...\n",
            "From: https://drive.google.com/uc?id=19-HG9CebZgVmLw_0yZMMeF9f6eoTqCYA\n",
            "To: /content/video.mp4\n",
            "\r0.00B [00:00, ?B/s]\r12.8MB [00:00, 199MB/s]\n"
          ],
          "name": "stdout"
        }
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "xk4VQaSSdy1N"
      },
      "source": [
        "Преобразовываем видео в последовательность картинок"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "n5fUonNt66f6",
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "outputId": "7cbb0f43-f07c-4eec-f3f5-36fd4028f1ee"
      },
      "source": [
        "%%time\n",
        "!ffmpeg -nostats -loglevel 0 -i '/content/video.mp4' -s 1280x720  '/content/images/out-%06d.jpg'\n",
        "clear_output()"
      ],
      "execution_count": 10,
      "outputs": [
        {
          "output_type": "stream",
          "text": [
            "CPU times: user 87 ms, sys: 14.9 ms, total: 102 ms\n",
            "Wall time: 17.7 s\n"
          ],
          "name": "stdout"
        }
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "Vw9n6Dzxd7ZJ"
      },
      "source": [
        "Детектим светофоры с помощью подгруженной ранее модели"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "zB-QoZu-7BA7",
        "outputId": "23ed852d-3265-4c8f-df44-e75b0db89ccb"
      },
      "source": [
        "%%time\n",
        "%cd /content/project/yolov3\n",
        "!python detect_custom.py --weights /content/model.pt --img 640 --source /content/images/ --save-dict > /dev/nul"
      ],
      "execution_count": 11,
      "outputs": [
        {
          "output_type": "stream",
          "text": [
            "/content/project/yolov3\n",
            "YOLOv3  torch 1.8.1+cu101 CUDA:0 (Tesla P4, 7611.9375MB)\n",
            "\n",
            "Model Summary: 261 layers, 61922845 parameters, 0 gradients, 156.3 GFLOPS\n",
            "CPU times: user 641 ms, sys: 85.8 ms, total: 727 ms\n",
            "Wall time: 2min 22s\n"
          ],
          "name": "stdout"
        }
      ]
    },
    {
      "source": [
        "Создаем вспомогательные переменные"
      ],
      "cell_type": "markdown",
      "metadata": {}
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "ltOYE8_JQrbq"
      },
      "source": [
        "frame_boxes_path = '/content/project/yolov3/runs/detect/exp/boxes.json'\n",
        "with open(frame_boxes_path) as json_file:\n",
        "    frame_boxes = json.load(json_file)"
      ],
      "execution_count": 12,
      "outputs": []
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "NQY0HHtfYaMR"
      },
      "source": [
        "img_folder = '/content/images'\n",
        "img_names = os.listdir(img_folder)\n",
        "img_names.sort()"
      ],
      "execution_count": 13,
      "outputs": []
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "zmm4F47EXzsE"
      },
      "source": [
        "result = {}\n",
        "keys = range(len(img_names))\n",
        "result = {key:{} for key in keys}"
      ],
      "execution_count": 14,
      "outputs": []
    },
    {
      "source": [
        "Определяем цвет и affect светофоры"
      ],
      "cell_type": "markdown",
      "metadata": {}
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "42YQLi0I1UEx"
      },
      "source": [
        "for img_path, boxes in frame_boxes.items():\n",
        "    j = 0\n",
        "    for box in boxes:\n",
        "        xywh = np.array(box)\n",
        "        img = cv2.imread(img_path)\n",
        "        h = img.shape[0]\n",
        "        w = img.shape[1]\n",
        "        x1 = round((xywh[0] - xywh[2] / 2) * w)\n",
        "        x2 = round((xywh[0] + xywh[2] / 2) * w)\n",
        "        y1 = round((xywh[1] - xywh[3] / 2) * h)\n",
        "        y2 = round((xywh[1] + xywh[3] / 2) * h)\n",
        "        img_crop = img[y1:y2, x1:x2]\n",
        "\n",
        "        color = detect_color(img_crop)\n",
        "        affect = affect_check([x1, y1, x2, y2], img_size=(w, h))\n",
        "\n",
        "        img_name = os.path.split(img_path)[-1]\n",
        "        frame_id = img_names.index(img_name)\n",
        "        result[frame_id][\"traffic_light_%d\" % j] = {'coords': [x1, y1, x2, y2],\n",
        "                                                    'state': color,\n",
        "                                                    'affect': affect\n",
        "                                                    }\n",
        "        j += 1"
      ],
      "execution_count": 15,
      "outputs": []
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "fOYdsWrvU54V"
      },
      "source": [
        "timestr = time.strftime(\"%d-%m %H:%M:%S\")\n",
        "json_path = '/content/drive/MyDrive/results/result '+ timestr\n",
        "with open(json_path  + '.json', 'w') as fp:\n",
        "    json.dump(result, fp, indent=4)"
      ],
      "execution_count": 16,
      "outputs": []
    },
    {
      "source": [
        "Производим постобработку результатов"
      ],
      "cell_type": "markdown",
      "metadata": {}
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "_bE3lw-tPsfv",
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "outputId": "f3734c47-6694-409d-cab2-f8b617b8d03d"
      },
      "source": [
        "path = json_path + '.json'\n",
        "\n",
        "with open(path) as json_file:\n",
        "    data = json.load(json_file)\n",
        "index = {}\n",
        "for i, value in enumerate(data):\n",
        "    index[i] = value\n",
        "\n",
        "data = pd.read_json(path, 'index')\n",
        "coord = []\n",
        "stat = []\n",
        "aff = []\n",
        "old = data.columns\n",
        "for column in data:\n",
        "    for light in data[column]:\n",
        "        try:\n",
        "            coord.append(light['coords'])\n",
        "            stat.append(light['state'])\n",
        "            aff.append(light['affect'])\n",
        "        except TypeError:\n",
        "            coord.append(None)\n",
        "            stat.append(None)\n",
        "            aff.append(None)\n",
        "    data[f'{column}_coords'] = coord\n",
        "    data[f'{column}_state'] = stat\n",
        "    data[f'{column}_affect'] = aff\n",
        "    coord = []\n",
        "    stat = []\n",
        "    aff = []\n",
        "data = data.drop(old, axis=1)\n",
        "data = data.replace({np.nan: None})\n",
        "\n",
        "for i in range(len(index)):\n",
        "    try:\n",
        "        data.loc[i]\n",
        "    except KeyError:\n",
        "        data.loc[i] = None\n",
        "data.sort_index(inplace=True)\n",
        "data = data.replace({np.nan: None})"
      ],
      "execution_count": 17,
      "outputs": [
        {
          "output_type": "stream",
          "text": [
            "/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: FutureWarning: Starting with Pandas version 2.0 all arguments of read_json except for the argument 'path_or_buf' will be keyword-only\n",
            "  if __name__ == '__main__':\n"
          ],
          "name": "stderr"
        }
      ]
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "TGvi4XAnP91U"
      },
      "source": [
        "window = 6\n",
        "tl_number = len(data.columns) // 3\n",
        "\n",
        "for tl in range(tl_number):\n",
        "    for i in range(len(data) - window):\n",
        "        past = data.iloc[i][0 + 3*tl]\n",
        "        future = data.iloc[i + 1][0 + 3*tl]\n",
        "        if check_coords(past, future) == None or check_coords(past, future) == False:\n",
        "            delta_result = []\n",
        "            for k in range(1, window):\n",
        "                delta = check_coords(data.iloc[i + k][0 + 3*tl], data.iloc[i + k + 1][0 + 3*tl])\n",
        "                delta_result.append(delta)\n",
        "            if np.count_nonzero(np.array(delta_result)) >= 1:\n",
        "                m = delta_result.index(True)\n",
        "                data.loc[i][data.columns[0 + 3*tl]] = data.loc[i + 1 + m][data.columns[0 + 3*tl]]\n",
        "                data.loc[i][data.columns[1 + 3*tl]] = data.loc[i + 1 + m][data.columns[1 + 3*tl]]\n",
        "                data.loc[i][data.columns[2 + 3*tl]] = data.loc[i + 1 + m][data.columns[2 + 3*tl]]\n",
        "            elif not np.array(delta_result).any() != None:\n",
        "                data.loc[i][data.columns[0 + 3*tl]] = None\n",
        "                data.loc[i][data.columns[1 + 3*tl]] = None\n",
        "                data.loc[i][data.columns[2 + 3*tl]] = None"
      ],
      "execution_count": 18,
      "outputs": []
    },
    {
      "source": [
        "Сохраняем наш json с предсказаниями"
      ],
      "cell_type": "markdown",
      "metadata": {}
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "SIi5w2AVP_q3"
      },
      "source": [
        "path_out = json_path + '_post.json'\n",
        "\n",
        "out = {}\n",
        "for row in data.iterrows():\n",
        "    frame_dict = {}\n",
        "    r = list(row[1])\n",
        "    for light in range(len(data.columns) // 3):\n",
        "        if r[0+3*light] is None:\n",
        "            break\n",
        "        else:\n",
        "            frame_dict[f'traffic_light_{light}'] = {}\n",
        "            frame_dict[f'traffic_light_{light}']['coords'] = r[0+3*light]\n",
        "            frame_dict[f'traffic_light_{light}']['state'] = r[1+3*light]\n",
        "            frame_dict[f'traffic_light_{light}']['affect'] = bool(r[2+3*light])\n",
        "    out[int(row[0])] = frame_dict\n",
        "\n",
        "with open(path_out, \"w\") as outfile: \n",
        "    json.dump(out, outfile, indent = 4)"
      ],
      "execution_count": 19,
      "outputs": []
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "UVyz-zTepRnv"
      },
      "source": [
        "КОНЕЦ!"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "7B-DyVDfk4If"
      },
      "source": [
        "Далее сделаем видео\n",
        "\n",
        "Размечаем картинки"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "w3DyDy8baRHK"
      },
      "source": [
        "result = out\n",
        "output_folder = '/content/output'\n",
        "for frame_id in result.keys():\n",
        "    img_path = os.path.join(img_folder, img_names[frame_id])\n",
        "    img = cv2.imread(img_path)\n",
        "    for tl in result[frame_id].values():\n",
        "        color = tl['state']\n",
        "        box = tl['coords']\n",
        "        affect = tl['affect']\n",
        "        label = color + ' (' + str(affect) + ')'\n",
        "        cv2.putText(img, label, (box[2], box[3]), \n",
        "                    cv2.FONT_HERSHEY_COMPLEX, \n",
        "                    fontScale=0.8, color=(128, 0, 128), thickness=2)\n",
        "        cv2.rectangle(img, (box[0], box[1]), (box[2], box[3]), \n",
        "                      color=(128, 0, 128), thickness=2)\n",
        "    path_out = os.path.join(output_folder, img_names[frame_id])\n",
        "    cv2.imwrite(path_out, img)"
      ],
      "execution_count": 20,
      "outputs": []
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "iCNf08-JebXc"
      },
      "source": [
        "Собираем видео из размеченных картинок "
      ]
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "KFrA8yOy53f8",
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "outputId": "4cebda7d-00b8-4d18-8009-55b9617169fd"
      },
      "source": [
        "%%time\n",
        "get_fps()\n",
        "%cd /content/drive/MyDrive/videos/\n",
        "!ffmpeg -nostats -loglevel 0 -y -framerate \"$(< /content/fps.txt)\" -i /content/output/out-%06d.jpg -c:v libx264 -crf 18 -pix_fmt yuv420p \"$(date)\"\".mp4\"\n",
        "clear_output()"
      ],
      "execution_count": 21,
      "outputs": [
        {
          "output_type": "stream",
          "text": [
            "CPU times: user 583 ms, sys: 62 ms, total: 645 ms\n",
            "Wall time: 1min 45s\n"
          ],
          "name": "stdout"
        }
      ]
    }
  ]
}

{'nbformat': 4,
 'nbformat_minor': 0,
 'metadata': {'colab': {'name': 'PROJECT_LAST.ipynb',
   'provenance': [],
   'collapsed_sections': []},
  'kernelspec': {'name': 'python3', 'display_name': 'Python 3'},
  'language_info': {'name': 'python'},
  'accelerator': 'GPU'},
 'cells': [{'cell_type': 'markdown',
   'metadata': {'id': 'cKYMitFbgsvS'},
   'source': ['Клонируем репу']},
  {'cell_type': 'code',
   'metadata': {'id': 'DVaeLq3Gg4hw'},
   'source': ['!sudo rm -rf /content/project'],
   'execution_count': 1,
   'outputs': []},
  {'cell_type': 'code',
   'metadata': {'colab': {'base_uri': 'https://localhost:8080/'},
    'id': 'o28E8GYVgphA',
    'outputId': '00fb8ba8-6382-4112-90d6-59ab06cb27ae'},
   'source': ['%cd /content\n',
    '!git clone https://github.com/STASVLAD/Tips-and-Tricks-in-DL project'],
   'execution_count': 2,
   'outputs': [{'output_type': 'stream',
     'text': ['/content\n',
      "Cloning into 'project'...\n",
      'remote: Enumerating objects: 264, done.\x1b